# Output versus pressure curve from 2020-02-24

In [ ]:
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy as sql
import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns
sns.set()
import getopt
import sys
import datetime
from scipy.interpolate import interp1d
from NGDataObject import NGDataObject
from scipy.optimize import curve_fit
from scipy import optimize

#*******************************
# CONSTANTS
#*******************************
# connection to database
CREDENTIALS_FILE = '/Users/hkromer/02_PhD/01.github/dash_NG/credentials.pw'
DB = "NG_twofast_DB" # name of the database
HOST = "twofast-RPi3-0"  # database host

# LOOKUP TABLES
LUT_PRESSURE_ION_SOURCE = "/Users/hkromer/02_PhD/01.github/phd/01_neutron_generator_contol/LUT_pressure_ion_source.txt"

In [ ]:
# reference detectors
# dose
# HV
DAY = "2020-02-24"
path = '/Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-02-24.Output_vs_pressure/'
# read password and user to connect to database
credentials = pd.read_csv(CREDENTIALS_FILE, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]


#*******************************
# GET DATA
#*******************************

# connect to DB
con = NGDataObject(host = HOST, database=DB, user=user, password=pw)

# get dose
query = "SELECT * FROM data_dose WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_dose = con.get_from_database(query=query)

# get HV
query = "SELECT * FROM data_HV WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_hv = con.get_from_database(query=query)
data_hv['HV_current_x100'] = data_hv['HV_current']*100.0

# get pressure
query = "SELECT * FROM data_pressure WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_pressure = con.get_from_database(query=query)

# get reference detectors
query = "SELECT * FROM data_referenceDetectors WHERE DATE(time) > '2020-02-03 20:00:00'" 
data_reference_full = con.get_from_database(query=query)

# get microwave power
query = "SELECT * FROM microwave_generator_power WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_microwave_power_full = con.get_from_database(query=query)

# get microwave frequency
query = "SELECT * FROM microwave_generator_frequency WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_microwave_frequency_full = con.get_from_database(query=query)

# save to file
data_reference_full['sum_counts'] = data_reference_full['counts_D1'] + data_reference_full['counts_D2'] + data_reference_full['counts_D3'] + data_reference_full['counts_D4'] 




In [ ]:
path = '/Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-02-24.Output_vs_pressure/'
DAY = "2020-02-24"

data_dose.to_csv(f'{path}/{DAY}_dose.csv')
data_hv.to_csv(f'{path}/{DAY}_hv.csv')
data_pressure.to_csv(f'{path}/{DAY}_pressure.csv')
# data_reference_full.to_csv(f'{path}/2020-02-07_reference_detectors.csv')
data_microwave_power_full.to_csv(f'{path}/{DAY}_microwave_power.csv')
data_microwave_frequency_full.to_csv(f'{path}/{DAY}_microwave_frequency.csv')

In [ ]:
path = '/Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-02-24.Output_vs_pressure/'
DAY = "2020-02-24"

data_dose = pd.read_csv(f'{path}/{DAY}_dose.csv', index_col=0)
data_dose.index = pd.to_datetime(data_dose.index)

data_hv = pd.read_csv(f'{path}/{DAY}_hv.csv', index_col=0)
data_hv.index = pd.to_datetime(data_hv.index)

data_pressure = pd.read_csv(f'{path}/{DAY}_pressure.csv', index_col=0)
data_pressure.index = pd.to_datetime(data_pressure.index)

# data_reference_full.to_csv(f'{path}/2020-02-07_reference_detectors.csv')
data_microwave_power_full = pd.read_csv(f'{path}/{DAY}_microwave_power.csv', index_col=0)
data_microwave_power_full.index = pd.to_datetime(data_microwave_power_full.index)

data_microwave_frequency_full = pd.read_csv(f'{path}/{DAY}_microwave_frequency.csv', index_col=0)
data_microwave_frequency_full.index = pd.to_datetime(data_microwave_frequency_full.index)



In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_dose.index, y='dose_corrected', data=data_dose, ax=ax, color='darkblue')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('Dose [muSv/hr]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv.index, y='HV_voltage', data=data_hv, ax=ax, color='darkred')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV V [kV]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv.index, y='HV_current', data=data_hv, ax=ax, color='darkorange')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax.set_xlabel(f'Time, {DAY}')
plt.show()


fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_pressure.index, y='pressure_IS_corrected', data=data_pressure, ax=ax, color='darkgreen')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('pressure [mbar]')
ax.set_xlabel(f'Time, {DAY}')
ax.set_yscale('log')
plt.show()


## Select time more closely when BROOKS was closed

- closed 13:28 Uhr

In [ ]:
time_start = f'{DAY} 13:28:00'
time_end = f'{DAY} 14:27:10'

df_hv = data_hv.loc[time_start:time_end]
df_pressure = data_pressure.loc[time_start:time_end]
df_dose = data_dose.loc[time_start:time_end]
df_freq = data_microwave_frequency_full.loc[time_start:time_end]

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_dose.index, y='dose_corrected', data=df_dose, ax=ax, color='darkblue')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('Dose [muSv/hr]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_hv.index, y='HV_voltage', data=df_hv, ax=ax, color='darkred')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV V [kV]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_hv.index, y='HV_current', data=df_hv, ax=ax, color='darkorange')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax.set_xlabel(f'Time, {DAY}')
plt.show()




fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_pressure.index, y='pressure_IS_corrected', data=df_pressure, ax=ax, color='darkgreen')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('pressure [mbar]')
ax.set_xlabel(f'Time, {DAY}')
ax.set_yscale('log')
plt.show()



fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_freq.index, y='frequency', data=df_freq, ax=ax, color='pink')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('Frequency [0.1 MHz]')
ax.set_xlabel(f'Time, {DAY}')
plt.show()


In [ ]:
# correlation between pressure read on display and pressure recorded in python
from sklearn.linear_model import LinearRegression

p_display = np.asarray([9.5e-6, 9.6e-6, 9.8e-6, 1.0e-5, 5.6e-6, 5.3e-6, 5.0e-6, 4.7e-6, 3.9e-6], float)
p_py = np.asarray([1.01e-5, 1.05e-5, 1.09e-5, 1.13e-5, 5.91e-6, 5.68e-6, 5.26e-6, 5.06e-6, 4.18e-6], float)

# convert into linear space
X = np.log(p_py).reshape(-1,1)
y = np.log(p_display).reshape(-1,1)

reg = LinearRegression().fit(X, y)
m = reg.coef_[0][0]
c = reg.intercept_[0]
y_fit = lambda x: x*m+c
plt.scatter(np.log(p_py), np.log(p_display))
x = np.linspace(-12.6, -11.2, 100)
plt.plot(x, y_fit(x), color='red')
plt.show()
reg.score(X, y), m, c

In [ ]:
def correct_display_pressure(row_pressure, y_fit):
    # conver to log
    p = np.log(row_pressure)
    # use correction
    p_out = y_fit(p)
    # transform back
    p_out = np.exp(p_out)
    
    return p_out
df_pressure.loc[:, 'pressure_IS_display_corr'] = df_pressure.loc[:, 'pressure_IS_corrected'].apply(lambda x: correct_display_pressure(x, y_fit))

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))

sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_pressure.index, y='pressure_IS_corrected', data=df_pressure, ax=ax, color='darkgreen')
sns.lineplot(x=df_pressure.index, y='pressure_IS_display_corr', data=df_pressure, ax=ax, color='darkred')

# plt.xlim(f'{DAY} 13:28:00', f'{DAY} 13:35:00')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('pressure [mbar]')
ax.set_xlabel(f'Time, {DAY}')
ax.set_yscale('log')
# plt.ylim(9.4e-6, 9.1e-6)
plt.show()

### Resample values to put into one dataframe

In [ ]:
def resample_30_seconds(df, cols, range_start, range_end):
    """
    Takes the a grouped df (grouped by day) and resamples the columns cols in 10s
    OUTPUT:
        - dataframe
    """

    d = {}

    s = pd.date_range(start=range_start, end=range_end, freq='10S')
    df_out = pd.DataFrame(pd.Series(s, name='time')).set_index('time')
    for col in cols:
        d[col] = df[col].resample('10S').mean()

    this_d = pd.DataFrame(d)

    df_out = df_out.merge(this_d, left_on=df_out.index, right_on=this_d.index, how='outer')
    df_out = df_out.set_index('key_0')
    df_out.index.name ='time'    
    
    return df_out

In [ ]:


# current resampled in 30s blocks
data_I = resample_30_seconds(df_hv, ['HV_current'], time_start, time_end)
# pressure resampled in 30s blocks
data_p = resample_30_seconds(df_pressure, ['pressure_IS_display_corr'], time_start, time_end)
data_freq = resample_30_seconds(df_freq, ['frequency'], time_start, time_end)

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_hv.index, y='HV_current', data=df_hv, ax=ax, color='darkorange')
sns.lineplot(x=data_I.index, y='HV_current', data=data_I, ax=ax, color='darkred')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax.set_xlabel(f'Time, {DAY}')
plt.show()

fig, ax = plt.subplots(figsize=(10,4))

sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=df_pressure.index, y='pressure_IS_display_corr', data=df_pressure, ax=ax, color='darkorange')
sns.lineplot(x=data_p.index, y='pressure_IS_display_corr', data=data_p, ax=ax, color='darkred')

# plt.xlim(f'{DAY} 13:28:00', f'{DAY} 13:35:00')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('pressure [mbar]')
ax.set_xlabel(f'Time, {DAY}')
ax.set_yscale('log')
# plt.ylim(9.4e-6, 9.1e-6)
plt.show()

In [ ]:
data = pd.merge(data_I, data_p, left_index=True, right_index=True, how='outer')
data = pd.merge(data, data_freq, left_index=True, right_index=True, how='outer')
data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
I = data['HV_current'].values # current in mA
p = data['pressure_IS_display_corr'].values * 1e6 # pressure in mbar

plt.scatter(p, I)

# ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax.set_xlabel(f'Pressure [1e6 mbar]')
# plt.xlim(0, 1e-5)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
I = data['HV_current'].values # current in mA
p = data['pressure_IS_display_corr'].values * 1e6 # pressure in mbar
freq = data['frequency'].values * 0.1 # frequency in MHz

plt.scatter(p, I, color='darkblue')
ax2 = ax.twinx()
ax2.plot(p, freq, color='darkorange')
# ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.set_ylabel('HV I [mA] (incl. leakage)')
ax2.set_ylabel('Frequency [MHz]')
ax.set_xlabel(f'Pressure [1e-6 mbar]')
# plt.xlim(0, 1e-5)
plt.show()

In [ ]:
freq